# scikit-learnとXGBoostの使い方

このノートでは、分類モデルの実験を「分割設計 → 交差検証 → チューニング → 最終評価」の順で実装します。
重要なのは、比較に使う設計を先に固定することです。モデルだけを入れ替えても、評価設計が曖昧だと結果を正しく解釈できません。

## 実行前準備

想定環境は Python 3.10 以上です。
未導入の場合は次を実行してください。

`%pip install scikit-learn xgboost pandas numpy matplotlib seaborn`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    StratifiedKFold,
    cross_validate,
    cross_val_predict,
    train_test_split,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)

try:
    from xgboost import XGBClassifier
    XGBOOST_AVAILABLE = True
except ModuleNotFoundError:
    XGBClassifier = None
    XGBOOST_AVAILABLE = False

sns.set_theme(style="whitegrid", context="notebook")
np.random.seed(42)


まずデータを確認します。
このデータでは `target=0` が malignant（悪性）、`target=1` が benign（良性）です。
このノートでは class 1（benign）を正例として Precision/Recall/F1 を計算します。

In [ ]:
cancer = load_breast_cancer(as_frame=True)
X = cancer.data
y = cancer.target

summary = pd.DataFrame({
    "n_samples": [len(X)],
    "n_features": [X.shape[1]],
    "benign_ratio(class1)": [float(y.mean())],
})
summary


In [ ]:
class_counts = y.value_counts().sort_index()
class_names = [cancer.target_names[i] for i in class_counts.index]

fig, ax = plt.subplots(figsize=(6, 3.4))
ax.bar(class_names, class_counts.values, color=["#4c78a8", "#f58518"])
ax.set_title("Class Balance")
ax.set_ylabel("count")
plt.tight_layout()
plt.show()


ここでデータを `dev`（開発用: 学習・CV・チューニングに使う）と `test`（最終確認専用）に分けます。
`test` は最後の1回だけ使い、探索中には触れません。

In [ ]:
X_dev, X_test, y_dev, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

print("dev shape :", X_dev.shape)
print("test shape:", X_test.shape)
print("dev  benign ratio:", round(float(y_dev.mean()), 3))
print("test benign ratio:", round(float(y_test.mean()), 3))


scikit-learn では Pipeline で前処理とモデルを一体化します。
これにより各 fold での前処理が訓練部分だけで fit され、リークを防ぎやすくなります。

In [ ]:
primary_metric = "roc_auc"
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

baseline_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=4000, random_state=42)),
])

baseline_cv = cross_validate(
    baseline_lr,
    X_dev,
    y_dev,
    cv=cv,
    scoring={"roc_auc": "roc_auc", "f1": "f1"},
    n_jobs=-1,
    return_train_score=False,
)

baseline_summary = {
    "roc_auc_mean": float(np.mean(baseline_cv["test_roc_auc"])),
    "roc_auc_std": float(np.std(baseline_cv["test_roc_auc"])),
    "f1_mean": float(np.mean(baseline_cv["test_f1"])),
    "f1_std": float(np.std(baseline_cv["test_f1"])),
}

pd.Series(baseline_summary).round(4)


In [ ]:
print("ROC-AUC mean/std:", round(baseline_summary["roc_auc_mean"], 4), "/", round(baseline_summary["roc_auc_std"], 4))
print("F1      mean/std:", round(baseline_summary["f1_mean"], 4), "/", round(baseline_summary["f1_std"], 4))
print("std が小さいほど、分割によるブレが小さく安定していると解釈できます。")


`cross_val_predict` で得る予測は OOF（Out-of-Fold）予測です。
各サンプルは、そのサンプルを学習に使っていないモデルで予測されます。

In [ ]:
baseline_oof_pred = cross_val_predict(
    baseline_lr,
    X_dev,
    y_dev,
    cv=cv,
    method="predict",
    n_jobs=-1,
)
cm = confusion_matrix(y_dev, baseline_oof_pred)

fig, ax = plt.subplots(figsize=(5, 4.2))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=False,
    ax=ax,
    xticklabels=class_names,
    yticklabels=class_names,
)
ax.set_title("Baseline Logistic (CV/OOF Confusion Matrix)")
ax.set_xlabel("Predicted")
ax.set_ylabel("Actual")
plt.tight_layout()
plt.show()


次に Logistic Regression をチューニングします。
Pipeline 内のパラメータは `<step名>__<パラメータ名>` で指定します。
たとえば `model__C` は LogisticRegression の `C` を意味します。
ここでは L1/L2 の両方を扱える `solver='liblinear'` を使います。

In [ ]:
lr_search = GridSearchCV(
    estimator=Pipeline([
        ("scaler", StandardScaler()),
        ("model", LogisticRegression(max_iter=5000, random_state=42, solver="liblinear")),
    ]),
    param_grid={
        "model__penalty": ["l1", "l2"],
        "model__C": [0.01, 0.1, 0.5, 1.0, 3.0, 10.0],
    },
    cv=cv,
    scoring=primary_metric,
    n_jobs=-1,
)
lr_search.fit(X_dev, y_dev)

print("best lr params:", lr_search.best_params_)
print(f"best lr cv {primary_metric}:", round(lr_search.best_score_, 4))


XGBoost でも同じ `dev` データと同じ CV 設定で探索します。
主なパラメータの意味は次の通りです。

- `max_depth`: 木の深さ。大きいほど複雑な境界を表現できます。
- `learning_rate`: 1本ごとの更新幅。小さくすると学習は遅いが安定しやすいです。
- `n_estimators`: 木の本数。多いほど表現力は上がるが過学習リスクも増えます。
- `min_child_weight`: 葉を分割するために必要な最小重み。大きいほど過学習を抑えます。

ここでは速度と探索範囲のバランスのため `RandomizedSearchCV(n_iter=20)` を使います。

In [ ]:
if XGBOOST_AVAILABLE:
    xgb_search = RandomizedSearchCV(
        estimator=XGBClassifier(
            eval_metric="logloss",
            random_state=42,
        ),
        param_distributions={
            "n_estimators": [150, 250, 400, 600],
            "max_depth": [2, 3, 4, 5, 6],
            "learning_rate": [0.01, 0.03, 0.05, 0.1],
            "subsample": [0.7, 0.8, 0.9, 1.0],
            "colsample_bytree": [0.6, 0.8, 1.0],
            "reg_lambda": [0.5, 1.0, 3.0, 10.0],
            "min_child_weight": [1, 3, 5],
        },
        n_iter=20,
        scoring=primary_metric,
        cv=cv,
        random_state=42,
        n_jobs=-1,
    )
    xgb_search.fit(X_dev, y_dev)

    print("best xgb params:", xgb_search.best_params_)
    print(f"best xgb cv {primary_metric}:", round(xgb_search.best_score_, 4))
else:
    xgb_search = None
    print("xgboost が未導入のため、XGBoost探索をスキップしました。")


候補モデルの比較は、テストではなく CV スコアで行います。
ここで勝者を決め、最後にその1モデルだけを `test` で評価します。

In [ ]:
candidates = {
    "Logistic (baseline)": baseline_summary[f"{primary_metric}_mean"],
    "Logistic (tuned)": float(lr_search.best_score_),
}
if xgb_search is not None:
    candidates["XGBoost (tuned)"] = float(xgb_search.best_score_)

cv_compare = pd.DataFrame({
    "model": list(candidates.keys()),
    f"cv_{primary_metric}": list(candidates.values()),
}).sort_values(f"cv_{primary_metric}", ascending=False)
cv_compare


In [ ]:
winner_name = cv_compare.iloc[0]["model"]
if winner_name == "Logistic (baseline)":
    final_model = baseline_lr
elif winner_name == "Logistic (tuned)":
    final_model = lr_search.best_estimator_
else:
    final_model = xgb_search.best_estimator_

final_model.fit(X_dev, y_dev)
final_pred = final_model.predict(X_test)
final_proba = final_model.predict_proba(X_test)[:, 1]

final_metrics = {
    "accuracy": accuracy_score(y_test, final_pred),
    "precision": precision_score(y_test, final_pred),
    "recall": recall_score(y_test, final_pred),
    "f1": f1_score(y_test, final_pred),
    "roc_auc": roc_auc_score(y_test, final_proba),
}

print("selected model:", winner_name)
pd.Series(final_metrics).round(4)


In [ ]:
cm_test = confusion_matrix(y_test, final_pred)
fig, ax = plt.subplots(figsize=(5, 4.2))
sns.heatmap(
    cm_test,
    annot=True,
    fmt="d",
    cmap="Greens",
    cbar=False,
    ax=ax,
    xticklabels=class_names,
    yticklabels=class_names,
)
ax.set_title(f"Final Model on Test ({winner_name})")
ax.set_xlabel("Predicted")
ax.set_ylabel("Actual")
plt.tight_layout()
plt.show()


In [ ]:
if xgb_search is not None and winner_name == "XGBoost (tuned)":
    importances = pd.Series(final_model.feature_importances_, index=X.columns).sort_values(ascending=False).head(12)
    fig, ax = plt.subplots(figsize=(7.2, 5))
    sns.barplot(x=importances.values, y=importances.index, orient="h", ax=ax, color="#4c78a8")
    ax.set_title("Top 12 Feature Importances (Final XGBoost)")
    ax.set_xlabel("importance")
    ax.set_ylabel("feature")
    plt.tight_layout()
    plt.show()
else:
    print("最終モデルがXGBoostではないか、xgboost未導入のため重要度表示をスキップしました。")


この流れで重要なのは、モデル比較に test を使わないことです。
比較は dev 内 CV で完結させ、test は最終確認に一度だけ使います。

scikit-learn は実験設計の再現性を作る土台として強く、XGBoost はその土台の上で精度を詰める有力候補です。